In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb

In [2]:
plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'legend.title_fontsize': 16,
          'legend.fontsize': 16,
          'legend.handlelength': 1,
          'figure.autolayout': 1,
          'font.size': 20,
          "figure.figsize": (6,4),
           'text.usetex' : True,
            'font.family' : 'lmodern'}
plt.rcParams.update(params)
plt.rcParams['text.usetex'] = True


/var/folders/1y/3mt7fkvd4s1bm0m2bp4fqyh00000gn/T/ipykernel_2423/2345625323.py:1: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]


In [3]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../python_scripts/')
import results_to_csvs

# Plots unfairness information gain and fairness

In [4]:
date = 'dt_9-9_10'
method = ['thresh']
fairs = ['DP', 'EODDS']
dsets = ['adult', 'default_credit','CrimeCommunity', 'COMPAS']
for meth in method:
    for dset in dsets:
        for fair in fairs:
            file_path = '../results_in_paper/{}_{}/{}_{}/overall_performance_seedwise.csv'.format(dset, fair, date, meth)
            IGS_path = '../results_in_paper/{}_{}/{}_{}/IGS.txt'.format(dset, fair, date, meth)
            igs = pd.read_csv(IGS_path, header=None)
            df = pd.read_csv(file_path, header=0)
            igs = igs.rename(columns={0: 'Threshold', 1:'IGS', 2: 'Depth'})
            #igs = igs.replace(1.7976931348623157e+308, np.nan)
            #igs = igs.dropna()
            threshes = igs['Threshold'].unique()
            #IGS_save = '../results_in_paper/{}_{}/{}_{}/IGS_depth.png'.format(dset, fair, date, meth)
            #IGS_v_depth(igs, IGS_save)
            IGS_save2 = '../IGS_thresh_plots/{}_{}_{}_IGS_thresh.pdf'.format(dset, fair, meth)
            IGS_unf_v_thresh(IGS_save2, igs, df, fair, meth, dset)
            #IGS_save3 = '../results_in_paper/{}_{}/{}_{}/IGS_unf.png'.format(dset, fair, date, meth)
            #IGS_v_unf(igs, df, fair, IGS_save3)

FileNotFoundError: [Errno 2] No such file or directory: '../results_in_paper/adult_DP/dt_9-9_10_thresh/IGS.txt'

In [9]:
def IGS_v_depth(igs, save_to):
    # igs = igs[igs['threshold']==.999]
    igs = igs.replace(1.7976931348623157e+308, np.nan)
    igs = igs.dropna()
    plt.clf()
    sb.lineplot(data=igs, x='Depth', y='IGS')
    plt.title('IGS vs. Depth')
    plt.savefig(save_to, bbox_inches='tight')

In [10]:
# plotting interploated IGS over unfairness
def IGS_v_unf(igs, df, fair, save_to):
    df['avgIGS'] = get_avg_IGS(igs)['avgIGS']
    frames = []
    y = 'avgIGS'
    if fair == 'DP':
        x = 'dp_diff'
    else:
        x = 'eodds_diff'

    # multiple y per x value
    plot_df = df[[x, y, 'threshold', 'seed']].replace([np.inf, -np.inf], np.nan)
    plot_df.columns = plot_df.columns[:2].tolist() + ['threshold', 'seed']
    plot_df.sort_values(by=['seed', x], inplace=True)

    # # INTERPOLATE DATA FOR PLOTTING
    all_x_values = plot_df[x].unique()
    seeds = plot_df['seed'].unique()
    for seed in seeds:
        per_seed_df = plot_df.loc[plot_df['seed'] == seed]
        xp = per_seed_df[x].values
        fp = per_seed_df[y].values

        x_values = sorted(list(set(all_x_values).difference(set(np.unique(xp)))))
        interpolated_df = {x: x_values, y: np.interp(x_values, xp, fp),
                           'seed': [seed] * len(x_values), 'threshold': [np.nan] * len(x_values)}
        plot_df = plot_df.append(pd.DataFrame.from_dict(interpolated_df), ignore_index=True)
    plot_df['Fairness type'] = [x] * len(plot_df)
    plot_df.sort_values(by=['seed', x], inplace=True)
    plot_df = plot_df.rename(columns={x: 'Unfairness'})
    #frames.append(plot_df)

    fig = plt.figure()
    plt.figure().clear()
    plt.close()
    plt.cla()
    plt.clf()
    new_x='Unfairness'

    sb.lineplot(data=plot_df[[new_x, y, 'seed', 'Fairness type']].sort_values(by=['seed', new_x]), 
                    x=new_x, y=y, estimator='mean', ci=95, color='b')
    #plt.title('Average IGS vs Unfairness')
    plt.savefig(save_to, bbox_inches='tight', dpi=1000)

In [11]:
# get average IGS value for each tree
def get_avg_IGS(igs):
    roots = igs.index[igs['Depth']==0]
    threshes = []
    IGS_avgs = []

    for i in range(len(roots)-1):
        tree = igs[roots[i]:roots[i+1]]
        tree = tree.replace(1.7976931348623157e+308, np.nan)
        tree.dropna(inplace=True)
        threshes.append(np.mean(tree['Threshold']))
        IGS_avgs.append(np.mean(tree['IGS']))
    avg_IGS = pd.DataFrame(np.asarray([threshes, IGS_avgs]).T, columns=['Threshold', 'avgIGS'])
    return avg_IGS

In [41]:
def IGS_unf_v_thresh(to_save, igs, df, fair, meth, dset):
    avg_IGS = get_avg_IGS(igs)
    fig, ax = plt.subplots()
    sb.lineplot(x="Threshold", y="avgIGS", data=avg_IGS, ax=ax, color='blue')
    g = ax.twinx()
    if fair=='DP':
        fairmeth = 'dp_diff'
    else:
        fairmeth = 'eodds_diff'
    sb.lineplot(x='threshold', y=fairmeth, data=df, color='red', ax=g)
    g.spines['right'].set_color('red')
    g.spines['left'].set_color('blue')
    if meth=='lambda':
        g.set_xlabel('Lambda')
        ax.set_xlabel('Lambda')
    else:
        g.set_xlabel(r'$\tau$')
        ax.set_xlabel(r'$\tau$')
    if fair=='DP':
        g.set_ylabel("Demographic Parity Gap")
        ax.set_ylabel(r"Unfairness $\textit{Gain}_{\textnormal{DP}}$")
    else:
        g.set_ylabel("Equalized Odds Gap")
        ax.set_ylabel(r"Unfairness $\textit{Gain}_{\textnormal{EOdds}}$")
    
    if dset == "COMPAS":
        g.set_ylim((0, .2))
        ax.set_ylim((0, .125))
    elif dset == "CrimeCommunity":
        ax.set_ylim((0, .2))
        g.set_ylim((0, .6))
    elif dset == "adult":
        g.set_ylim((0, .13))
        ax.set_ylim((0, .1))
    else:
        g.set_ylim((0, .06))
        ax.set_ylim((0, .06))
    #ax.set_ylabel("Average Unfairness Gain")

    if meth=='lambda':
        plt.xlim(left=.02)
    else:
        plt.xlim(left=.004)
    
    plt.savefig(to_save, bbox_inches='tight', dpi=300)